In [19]:
!pip install torchutils

In [20]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms as T

# Для чтения изображений с диска
from torchvision import io # input/output
import torchutils as tu
import json
import numpy as np
import matplotlib.pyplot as plt

In [21]:
config = {
    'batch_size': 512,
    'learning_rate': 0.003,
    'epochs': 15,
    'device': 'mps',
    'data_dir': 'data/',
    'log_dir': 'runs/experiment1'
}

DEVICE = config['device']

In [22]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jehanbhathena/weather-dataset")

print("Path to dataset files:", path)

Path to dataset files: /Users/dmitrijpavlov/.cache/kagglehub/datasets/jehanbhathena/weather-dataset/versions/3


In [23]:
import os
import shutil
from sklearn.model_selection import train_test_split

src_root = '/Users/dmitrijpavlov/.cache/kagglehub/datasets/jehanbhathena/weather-dataset/versions/3/dataset'
base_dir = os.path.dirname(src_root)
train_dir = os.path.join(base_dir, 'weather_train')
valid_dir = os.path.join(base_dir, 'weather_valid')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)

for class_name in os.listdir(src_root):
    class_path = os.path.join(src_root, class_name)
    if not os.path.isdir(class_path):
        continue

    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(valid_dir, class_name), exist_ok=True)

    images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
    
    train_img, valid_img = train_test_split(images, test_size=0.2, random_state=42, shuffle=True)

    for img in train_img:
        shutil.copy(os.path.join(class_path, img), os.path.join(train_dir, class_name, img))
    for img in valid_img:
        shutil.copy(os.path.join(class_path, img), os.path.join(valid_dir, class_name, img))

print(f"Train dataset: {train_dir}")
print(f"Validation dataset: {valid_dir}")

Train dataset: /Users/dmitrijpavlov/.cache/kagglehub/datasets/jehanbhathena/weather-dataset/versions/3/weather_train
Validation dataset: /Users/dmitrijpavlov/.cache/kagglehub/datasets/jehanbhathena/weather-dataset/versions/3/weather_valid


In [24]:
temp_trnsfrm = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])

temp_dataset = torchvision.datasets.ImageFolder('/Users/dmitrijpavlov/.cache/kagglehub/datasets/jehanbhathena/weather-dataset/versions/3/weather_valid', transform=temp_trnsfrm)
temp_loader = DataLoader(temp_dataset, batch_size=config['batch_size'], shuffle=False)

import torch

def compute_mean_std(loader):
    mean = torch.zeros(3)
    std = torch.zeros(3)
    total_images = 0

    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images += batch_samples

    mean /= total_images
    std /= total_images
    return mean, std

mean, std = compute_mean_std(temp_loader)
print(f'Mean: {mean}')
print(f'Std:  {std}')

Mean: tensor([0.5177, 0.5280, 0.5060])
Std:  tensor([0.1890, 0.1825, 0.1901])


In [25]:
train_trans = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(degrees=10),
    T.ToTensor(),
    T.Normalize(mean=[0.5169, 0.5246, 0.5059], std=[0.1900, 0.1839, 0.1903])
])

valid_trans = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.5169, 0.5246, 0.5059], std=[0.1900, 0.1839, 0.1903])
])

train_dataset = torchvision.datasets.ImageFolder('/Users/dmitrijpavlov/.cache/kagglehub/datasets/jehanbhathena/weather-dataset/versions/3/weather_train',
                                                 transform=train_trans)


valid_dataset = torchvision.datasets.ImageFolder('/Users/dmitrijpavlov/.cache/kagglehub/datasets/jehanbhathena/weather-dataset/versions/3/weather_valid',
                                                 transform=valid_trans)

train_dataset

Dataset ImageFolder
    Number of datapoints: 5484
    Root location: /Users/dmitrijpavlov/.cache/kagglehub/datasets/jehanbhathena/weather-dataset/versions/3/weather_train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
               Normalize(mean=[0.5169, 0.5246, 0.5059], std=[0.19, 0.1839, 0.1903])
           )

In [26]:
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True)

In [27]:
from torchvision.models import efficientnet_v2_m, EfficientNet_V2_M_Weights

weights = EfficientNet_V2_M_Weights.DEFAULT
model = efficientnet_v2_m(weights=weights).to(DEVICE)
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

In [28]:
model.classifier[1] = nn.Linear(1280, 11)

In [29]:
for param in model.parameters():
    param.requires_grad = False

In [30]:
for layer in model.features[-3:]:
    for param in layer.parameters():
        param.requires_grad = True

for param in model.classifier[1].parameters():
    param.requires_grad = True

In [33]:
model.to(DEVICE)
tu.get_model_summary(model, torch.randn(32, 3, 224, 224, device=DEVICE))

Layer                                                      Kernel               Output          Params             FLOPs
0_features.0.Conv2d_0                                     [3, 24, 3, 3]   [32, 24, 112, 112]         648     260,112,384
1_features.0.BatchNorm2d_1                                         [24]   [32, 24, 112, 112]          48      38,535,168
2_features.0.SiLU_2                                                   -   [32, 24, 112, 112]           0               0
3_features.1.0.block.0.Conv2d_0                          [24, 24, 3, 3]   [32, 24, 112, 112]       5,184   2,080,899,072
4_features.1.0.block.0.BatchNorm2d_1                               [24]   [32, 24, 112, 112]          48      38,535,168
5_features.1.0.block.0.SiLU_2                                         -   [32, 24, 112, 112]           0               0
6_features.1.0.StochasticDepth_stochastic_depth                       -   [32, 24, 112, 112]           0               0
7_features.1.1.block.0.Conv2d_0 

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config['learning_rate'])

In [35]:
def compute_batch_accuracy(preds, labels):
    predicted_classes = preds.argmax(dim=1)           
    correct = (predicted_classes == labels).sum()     
    accuracy = correct.float() / labels.size(0)      
    return accuracy.item()

In [ ]:
model.to(DEVICE)

train_loss_hist = []
valid_loss_hist = []
train_acc_hist = []
valid_acc_hist = []

for epoch in range(5):

    loss_batch = []
    acc_batch = []

    model.train()
    for images, labels in train_loader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        preds = model(images)

        loss = criterion(preds, labels)

        loss_batch.append(loss.item())
        acc_batch.append(compute_batch_accuracy(preds, labels))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss_hist.append(np.mean(loss_batch))
    train_acc_hist.append(np.mean(acc_batch))

    model.eval()
    loss_batch = []
    acc_batch  = []

    with torch.no_grad():
        for images, labels in valid_loader:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            preds = model(images)

            loss = criterion(preds, labels)
            loss_batch.append(loss.item())
            acc_batch.append(compute_batch_accuracy(preds, labels))


    valid_loss_hist.append(np.mean(loss_batch))
    valid_acc_hist.append(np.mean(acc_batch))
    print(f'[Epoch {epoch:02d}] Train loss: {train_loss_hist[-1]:.4f}, valid loss = {valid_loss_hist[-1]:.4f} Train acc {train_acc_hist[-1]:.4f} Valid acc {valid_acc_hist[-1]:.4f}')

fig, ax = plt.subplots(1, 2, figsize=(20, 4))
ax[0].plot(train_loss_hist)
ax[0].plot(valid_loss_hist)
all_losses = train_loss_hist + valid_loss_hist
ax[0].set_ylim((0, max(all_losses)))
ax[0].set_title('Loss history')

ax[1].plot(train_acc_hist)
ax[1].plot(valid_acc_hist)
ax[1].set_ylim(0, 1.1)
ax[1].set_title('Acc history');

In [ ]:
torch.save(model.state_dict(), "efficientnet_model.pth")

In [ ]:
import matplotlib.pyplot as plt

# Настройка стиля
plt.rcParams.update({
    'font.size': 12,
    'axes.titlesize': 16,
    'axes.labelsize': 14,
    'legend.fontsize': 12,
    'figure.titlesize': 16,
    'grid.linewidth': 0.5,
    'lines.linewidth': 2.5
})

fig, ax = plt.subplots(1, 2, figsize=(18, 5))

# Цвета
train_color = '#1f77b4'  # синий (стандартный, но аккуратный)
valid_color = '#ff7f0e'  # оранжевый

# --- Loss ---
ax[0].plot(train_loss_hist, label='Train Loss', color=train_color)
ax[0].plot(valid_loss_hist, label='Validation Loss', color=valid_color)
ax[0].set_title('Training and Validation Loss', fontsize=16, pad=15)
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Loss')
ax[0].legend(frameon=False)
ax[0].grid(True, linestyle='--', alpha=0.6)
ax[0].set_ylim(0, max(train_loss_hist + valid_loss_hist) * 1.05)

# --- Accuracy ---
ax[1].plot(train_acc_hist, label='Train Accuracy', color=train_color)
ax[1].plot(valid_acc_hist, label='Validation Accuracy', color=valid_color)
ax[1].set_title('Training and Validation Accuracy', fontsize=16, pad=15)
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Accuracy')
ax[1].set_ylim(0, 1.02)
ax[1].legend(frameon=False)
ax[1].grid(True, linestyle='--', alpha=0.6)

# Улучшение макета
plt.tight_layout(pad=3.0)

# Сохранение в высоком качестве
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')

# Показать график (опционально)
plt.show()